# Run this first to create a master label to ensure accurate labeling accross all data sets


# Using 

  1. https://www.kaggle.com/datasets/jessicali9530/stanford-dogs-dataset


In [1]:
# set up some imports
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pandas as pd
# import a progress bar using tqdm
from tqdm.auto import tqdm
from pathlib import Path
import numpy as np
import requests
import zipfile
import random
import os
import shutil
from PIL import Image
from typing import Tuple, Dict, List
from torchinfo import summary
from timeit import default_timer as timer
import re

# set device to gpu if possible
targeted_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\thebl\OneDrive\Documents\Projects\dog-learn-projects\dev-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_data(data_url: str, zip_name: str) -> None:
  data_path = Path("data")
  zip_file_path = Path(f"{data_path}/{zip_name}.zip")
  img_path = data_path / zip_name

  if not zip_file_path.exists():
    with requests.get(data_url, stream=True) as r:
      r.raise_for_status()
      total_size = int(r.headers.get('content-length', 0))
      block_size = 1024
      with open(zip_file_path, "wb") as f, tqdm(
        total=total_size, unit='iB', unit_scale=True, desc="Downloading",
      ) as bar:
        for chunk in r.iter_content(chunk_size=block_size):
          f.write(chunk)
          bar.update(len(chunk))
  else: 
    print("File already exists, skipping download")
  
  # unzip the file 
  with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    print("unzipping files")
    
    # extract all the files in the zip in the image path
    zip_ref.extractall(img_path)


In [3]:
url_list = [
  "https://www.kaggle.com/api/v1/datasets/download/jessicali9530/stanford-dogs-dataset"
]

In [4]:
create_data(url_list[0], "dogs_data")

File already exists, skipping download
unzipping files


In [5]:
def extract_basic_folders(base_dir: str):
  animal_labels = []
  for animal_folder in tqdm(base_dir.iterdir()):
     if animal_folder.is_dir():
        animal_labels.append(animal_folder.name)
  return animal_labels

In [6]:
def lower_case_folders(folder_dir, names:list) -> list:
  for i in range(len(names)):
    current_folder = Path(f"{folder_dir}/{names[i]}")
    names[i] = names[i].lower().replace('_', ' ')
    
    # Create the new path
    new_folder = folder_dir / names[i]
    current_folder.rename(new_folder)
  return names

In [7]:
def clean_folder_name_v9(folder_dir, names:list) -> list:
  for i in range(len(names)):  
    current_folder = Path(f"{folder_dir}/{names[i]}")

    parts = names[i].split('-', 1)
    if len(parts) == 2:
      names[i] = parts[1].replace('_', ' ').lower()
    else:
      names[i] = names[i].lower()
    
    # Create the new path
    new_folder = folder_dir / names[i]
    current_folder.rename(new_folder)

  return names
  

In [9]:
# create a master label list for all three datasets

# special processing for names from n02085620-Chihuahua to chihuahua
source_dogs_v1 = Path('data/dogs_data/images/Images')


In [11]:
result = extract_basic_folders(source_dogs_v1)
result = clean_folder_name_v9(source_dogs_v1, result)
result

120it [00:00, 8532.81it/s]


['chihuahua',
 'japanese spaniel',
 'maltese dog',
 'pekinese',
 'shih-tzu',
 'blenheim spaniel',
 'papillon',
 'toy terrier',
 'rhodesian ridgeback',
 'afghan hound',
 'basset',
 'beagle',
 'bloodhound',
 'bluetick',
 'black-and-tan coonhound',
 'walker hound',
 'english foxhound',
 'redbone',
 'borzoi',
 'irish wolfhound',
 'italian greyhound',
 'whippet',
 'ibizan hound',
 'norwegian elkhound',
 'otterhound',
 'saluki',
 'scottish deerhound',
 'weimaraner',
 'staffordshire bullterrier',
 'american staffordshire terrier',
 'bedlington terrier',
 'border terrier',
 'kerry blue terrier',
 'irish terrier',
 'norfolk terrier',
 'norwich terrier',
 'yorkshire terrier',
 'wire-haired fox terrier',
 'lakeland terrier',
 'sealyham terrier',
 'airedale',
 'cairn',
 'australian terrier',
 'dandie dinmont',
 'boston bull',
 'miniature schnauzer',
 'giant schnauzer',
 'standard schnauzer',
 'scotch terrier',
 'tibetan terrier',
 'silky terrier',
 'soft-coated wheaten terrier',
 'west highland wh

In [12]:
master_folder = Path("data/cleaned_dogs_data_filtered")
master_folder.mkdir(parents=True, exist_ok=True)

In [11]:
# No image filtering 
'''
all_dogs_sources = [
    source_dogs_v1
]

# Iterate over each source folder
for source_folder in tqdm(all_dogs_sources, desc="Processing source folders"):
    for animal_folder in source_folder.iterdir():
        if animal_folder.is_dir():
            # Create a corresponding folder in the master directory
            new_folder = master_folder / animal_folder.name
            new_folder.mkdir(exist_ok=True)
            
            # Iterate over image files in the current animal folder
            for img_file in tqdm(animal_folder.iterdir(), desc=f"Processing {animal_folder.name}"):
                if img_file.is_file():
                    try:
                        # Open the image to check its size
                        with Image.open(img_file) as img:
                          shutil.copy(img_file, new_folder / img_file.name)
                    except Exception as e:
                        print(f"Error processing {img_file.name}: {e}")

                        '''


'\nall_dogs_sources = [\n    source_dogs_v1\n]\n\n# Iterate over each source folder\nfor source_folder in tqdm(all_dogs_sources, desc="Processing source folders"):\n    for animal_folder in source_folder.iterdir():\n        if animal_folder.is_dir():\n            # Create a corresponding folder in the master directory\n            new_folder = master_folder / animal_folder.name\n            new_folder.mkdir(exist_ok=True)\n\n            # Iterate over image files in the current animal folder\n            for img_file in tqdm(animal_folder.iterdir(), desc=f"Processing {animal_folder.name}"):\n                if img_file.is_file():\n                    try:\n                        # Open the image to check its size\n                        with Image.open(img_file) as img:\n                          shutil.copy(img_file, new_folder / img_file.name)\n                    except Exception as e:\n                        print(f"Error processing {img_file.name}: {e}")\n\n                    

In [13]:
# Filterd Version
all_dogs_sources = [
    source_dogs_v1
]

# Iterate over each source folder
for source_folder in tqdm(all_dogs_sources, desc="Processing source folders"):
    for animal_folder in source_folder.iterdir():
        if animal_folder.is_dir():
            # Create a corresponding folder in the master directory
            new_folder = master_folder / animal_folder.name
            new_folder.mkdir(exist_ok=True)
            
            # Iterate over image files in the current animal folder
            for img_file in tqdm(animal_folder.iterdir(), desc=f"Processing {animal_folder.name}"):
                if img_file.is_file():
                    try:
                        # Open the image to check its size
                        with Image.open(img_file) as img:
                            width, height = img.size
                            # Proceed only if both dimensions are at least 300 pixels
                            if width >= 224 and height >= 224:
                                # Copy the image to the new folder
                                shutil.copy(img_file, new_folder / img_file.name)
                            else:
                                print(f"Skipping {img_file.name}: dimensions {width}x{height} are smaller than 224x224.")
                    except Exception as e:
                        print(f"Error processing {img_file.name}: {e}")


Processing source folders:   0%|          | 0/1 [00:00<?, ?it/s]

Skipping n02110627_11211.jpg: dimensions 200x156 are smaller than 224x224.
Skipping n02110627_11365.jpg: dimensions 109x110 are smaller than 224x224.
Skipping n02110627_11422.jpg: dimensions 160x170 are smaller than 224x224.
Skipping n02110627_11435.jpg: dimensions 300x199 are smaller than 224x224.
Skipping n02110627_11584.jpg: dimensions 200x150 are smaller than 224x224.
Skipping n02110627_11614.jpg: dimensions 216x152 are smaller than 224x224.
Skipping n02110627_11783.jpg: dimensions 200x200 are smaller than 224x224.
Skipping n02110627_11798.jpg: dimensions 225x175 are smaller than 224x224.
Skipping n02110627_11819.jpg: dimensions 250x186 are smaller than 224x224.
Skipping n02110627_11835.jpg: dimensions 240x180 are smaller than 224x224.
Skipping n02110627_11858.jpg: dimensions 198x187 are smaller than 224x224.
Skipping n02110627_11875.jpg: dimensions 142x142 are smaller than 224x224.
Skipping n02110627_11942.jpg: dimensions 200x280 are smaller than 224x224.
Skipping n02110627_12070.

Skipping n02110627_1966.jpg: dimensions 212x257 are smaller than 224x224.
Skipping n02110627_2157.jpg: dimensions 132x200 are smaller than 224x224.
Skipping n02110627_233.jpg: dimensions 171x200 are smaller than 224x224.
Skipping n02110627_2894.jpg: dimensions 186x207 are smaller than 224x224.
Skipping n02110627_3026.jpg: dimensions 170x150 are smaller than 224x224.
Skipping n02110627_4597.jpg: dimensions 128x150 are smaller than 224x224.
Skipping n02110627_5743.jpg: dimensions 200x305 are smaller than 224x224.
Skipping n02110627_6414.jpg: dimensions 250x197 are smaller than 224x224.
Skipping n02110627_6991.jpg: dimensions 200x150 are smaller than 224x224.
Skipping n02110627_7013.jpg: dimensions 200x200 are smaller than 224x224.
Skipping n02110627_735.jpg: dimensions 150x160 are smaller than 224x224.


Processing affenpinscher: 150it [00:00, 268.16it/s]


Skipping n02110627_8250.jpg: dimensions 112x120 are smaller than 224x224.
Skipping n02110627_8621.jpg: dimensions 270x222 are smaller than 224x224.
Skipping n02110627_8695.jpg: dimensions 200x140 are smaller than 224x224.
Skipping n02110627_8714.jpg: dimensions 200x200 are smaller than 224x224.


...

Processing wire-haired fox terrier: 157it [00:00, 278.66it/s]


Skipping n02095314_3509.jpg: dimensions 250x209 are smaller than 224x224.
Skipping n02095314_3527.jpg: dimensions 200x133 are smaller than 224x224.
Skipping n02095314_398.jpg: dimensions 292x202 are smaller than 224x224.
Skipping n02095314_448.jpg: dimensions 180x135 are smaller than 224x224.
Skipping n02095314_464.jpg: dimensions 250x204 are smaller than 224x224.
Skipping n02095314_550.jpg: dimensions 163x190 are smaller than 224x224.
Skipping n02095314_662.jpg: dimensions 206x227 are smaller than 224x224.
Skipping n02095314_669.jpg: dimensions 201x288 are smaller than 224x224.
Skipping n02095314_80.jpg: dimensions 340x220 are smaller than 224x224.
Skipping n02095314_928.jpg: dimensions 400x199 are smaller than 224x224.
Skipping n02095314_963.jpg: dimensions 298x202 are smaller than 224x224.


Skipping n02094433_2776.jpg: dimensions 190x259 are smaller than 224x224.
Skipping n02094433_2838.jpg: dimensions 170x250 are smaller than 224x224.
Skipping n02094433_3198.jpg: dimensions 202x250 are smaller than 224x224.
Skipping n02094433_3640.jpg: dimensions 200x150 are smaller than 224x224.
Skipping n02094433_3905.jpg: dimensions 248x179 are smaller than 224x224.
Skipping n02094433_478.jpg: dimensions 250x188 are smaller than 224x224.


Skipping n02094433_5350.jpg: dimensions 300x200 are smaller than 224x224.
Skipping n02094433_6328.jpg: dimensions 265x173 are smaller than 224x224.
Skipping n02094433_7394.jpg: dimensions 154x200 are smaller than 224x224.


Processing yorkshire terrier: 164it [00:00, 213.14it/s]
Processing source folders: 100%|██████████| 1/1 [01:47<00:00, 107.39s/it]
